In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()


,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.063611,True,32,1,37.0
1,4.687028,True,30,3,33.0
2,7.561856,True,20,1,39.0
3,7.561856,True,31,1,37.0
4,7.312733,True,32,1,40.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9997.000000,10000.000000,10000.000000,9886.000000
mean,7.303938,27.226500,1.033500,38.700890
std,1.313289,6.194646,0.192825,2.538734
min,0.595248,13.000000,1.000000,18.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.126239,32.000000,1.000000,40.000000
max,12.625874,52.000000,4.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5096
False    4904
Name: is_male, dtype: int64

In [5]:
df.isnull().sum()

weight_pounds        3
is_male              0
mother_age           0
plurality            0
gestation_weeks    114
dtype: int64

In [6]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [7]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [8]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
40,0,18,1,39.0
5976,0,22,1,47.0
6902,0,29,1,38.0
4838,0,32,1,37.0
8335,1,29,1,39.0


In [9]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [10]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

2021-10-21 05:58:08.272477: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200160000 Hz
2021-10-21 05:58:08.273678: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555680110750 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-21 05:58:08.273710: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-21 05:58:08.273982: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6671 samples, validate on 742 samples
Epoch 1/10
6671/6671 [==============================] - 2s 241us/sample - loss: 7.2161 - mae: 1.4624 - mse: 7.2161 - val_loss: 2.0252 - val_mae: 1.1507 - val_mse: 2.0252
Epoch 2/10
6671/6671 [==============================] - 1s 114us/sample - loss: 1.4628 - mae: 0.9448 - mse: 1.4628 - val_loss: 1.2509 - val_mae: 0.8805 - val_mse: 1.2509
Epoch 3/10
6671/6671 [==============================] - 1s 118us/sample - loss: 1.4238 - mae: 0.9317 - mse: 1.4238 - val_loss: 1.6492 - val_mae: 1.0167 - val_mse: 1.6492
Epoch 4/10
6671/6671 [==============================] - 1s 120us/sample - loss: 1.3799 - mae: 0.9156 - mse: 1.3799 - val_loss: 1.3134 - val_mae: 0.8889 - val_mse: 1.3134
Epoch 5/10
6671/6671 [==============================] - 1s 111us/sample - loss: 1.3431 - mae: 0.9040 - mse: 1.3431 - val_loss: 1.3011 - val_mae: 0.9077 - val_mse: 1.3011
Epoch 6/10
6671/6671 [==============================] - 1s 124us/sample - loss: 1.3230 - mae: 0.8936 - 

In [14]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])


In [15]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.0445313
Actual val:  6.18617107172

Predicted val:  7.639707
Actual val:  7.3193470984

Predicted val:  7.429815
Actual val:  7.06361087448

Predicted val:  7.5500684
Actual val:  7.12534030784

Predicted val:  7.696812
Actual val:  7.68751907594

Predicted val:  7.8699474
Actual val:  8.75014717878

Predicted val:  7.3946967
Actual val:  7.43839671988

Predicted val:  6.4711967
Actual val:  5.7761112644

Predicted val:  7.7040515
Actual val:  7.0768386102

Predicted val:  7.744945
Actual val:  8.24969784404



In [16]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [17]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [18]:
#config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  #.set_custom_predict_fn(custom_predict)
  #.set_target_feature('weight_pounds')
  #.set_model_type('regression'))
#WitWidget(config_builder, height=800)